# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   600.270432 -0.052258  1234  600.218174  600.322689  1200.436347   
6   876.306316  0.198105  1234  876.504421  876.108212  1753.008843   
7   731.800901  0.551890  1234  732.352791  731.249011  1464.705581   
8   249.774468  0.980017  1234  250.754485  248.794452   501.508970   
9   839.668021  0.377748  1234  840.045770  839.290273  1680.091540   
10  659.826802 -0.632565  1234  659.194237  660.459366  1318.388474   
11  959.407427 -0.002880  1234  959.404546  959.410307  1918.809093   
12  360.646866 -0.700581  1234  359.946285  361.347448   719.892570   
13  293.350595 -0.092911  1234  293.257684  293.443506   586.515368   
14  349.155894  0.379057  1234  349.534951  348.776837   699.069901   

              f  
5   1200.645379  
6   1752.216423  
7   1462.498022  
8    497.588903  
9   1678.580546  
10  1320.918733  
11  1918.820614  
12   722.694896  
13   586.887013  
14   697.553674

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -600.218174   600.322689
6   -876.504421   876.108212
7   -732.352791   731.249011
8   -250.754485   248.794452
9   -840.045770   839.290273
10  -659.194237   660.459366
11  -959.404546   959.410307
12  -359.946285   361.347448
13  -293.257684   293.443506
14  -349.534951   348.776837

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -600.218174   600.322689
6   -876.504421   876.108212
7   -732.352791   731.249011
8   -250.754485   248.794452
9   -840.045770   839.290273
10  -659.194237   660.459366
11  -959.404546   959.410307
12  -359.946285   361.347448
13  -293.257684   293.443506
14  -349.534951   348.776837

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-600.2181737 ,  600.32268948],
       [-876.50442128,  876.10821158],
       [-732.35279073,  731.24901094],
       [-250.75448512,  248.79445162],
       [-840.04576998,  839.29027299],
       [-659.19423681,  660.4593664 ],
       [-959.40454632,  959.41030707],
       [-359.94628482,  361.3474478 ],
       [-293.25768401,  293.4435065 ],
       [-349.53495058,  348.77683715]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1198.436347  -1200.645379
6    1751.008843  -1752.216423
7    1462.705581  -1462.498022
8     499.508970   -497.588903
9    1678.091540  -1678.580546
10   1316.388474  -1320.918733
11   1916.809093  -1918.820614
12    717.892570   -722.694896
13    584.515368   -586.887013
14    697.069901   -697.553674

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf